# Generador de Explicaciones para Universidades

Este notebook utiliza un modelo de lenguaje (LLM) para generar explicaciones detalladas sobre por qué cada universidad ha recibido determinadas calificaciones en sus métricas académicas.

## Funcionalidades:
1. Recalcular campos seleccionados para todas las universidades
2. Generar explicaciones mediante LLM para las métricas académicas
3. Almacenar las explicaciones en un nuevo campo `metadata.descripcion`
4. Visualizar las explicaciones generadas

## Configuración Inicial

Primero importamos las librerías necesarias y configuramos el entorno:

In [ ]:
import os
import sys
import json
import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
from bson.objectid import ObjectId
from IPython.display import display, HTML

# Asegurarnos de que podemos importar nuestro módulo de utilidades
sys.path.append('./utils')
from data_explainer import recalculate_fields, get_university_explanations, connect_mongodb

# Cargar variables de entorno desde .env
load_dotenv()

# Funciones para mensajes estilizados
def success_msg(msg):
    display(HTML(f'<div style="background-color: #d4edda; color: #155724; padding: 10px; border-radius: 5px; margin: 10px 0;">{msg}</div>'))

def error_msg(msg):
    display(HTML(f'<div style="background-color: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px; margin: 10px 0;">{msg}</div>'))

def info_msg(msg):
    display(HTML(f'<div style="background-color: #cce5ff; color: #004085; padding: 10px; border-radius: 5px; margin: 10px 0;">{msg}</div>'))

## Conexión a MongoDB

Conectamos a la base de datos para poder acceder a la información de los programas:

In [ ]:
# Conectar a MongoDB
mongo_client, db = connect_mongodb()

# Verificar la conexión
if mongo_client and db:
    success_msg(f"Conexión exitosa a MongoDB: {db.name}")
    
    # Obtener estadísticas básicas
    num_universidades = db.programas.distinct("universidad")
    total_programas = db.programas.count_documents({})
    info_msg(f"Base de datos contiene {len(num_universidades)} universidades con un total de {total_programas} programas de doctorado.")
else:
    error_msg("No se pudo conectar a MongoDB. Verifica la variable de entorno MONGODB_URI.")

## Análisis de las Métricas Actuales

Antes de generar nuevas explicaciones, veamos qué métricas están disponibles actualmente:

In [ ]:
# Obtener un resumen de las métricas disponibles por universidad
def get_universities_stats():
    universities_stats = db.programas.aggregate([
        {
            "$group": {
                "_id": "$universidad",
                "stats": {"$first": "$stats"},
                "has_metadata": {"$sum": {"$cond": [{"$ifNull": ["$metadata.descripcion", False]}, 1, 0]}},
                "programa_count": {"$sum": 1}
            }
        },
        {
            "$project": {
                "_id": 0,
                "universidad": "$_id",
                "stats": 1,
                "has_metadata": 1,
                "programa_count": 1
            }
        }
    ])
    
    return list(universities_stats)

# Convertir a DataFrame para visualización
stats_df = pd.DataFrame(get_universities_stats())

if not stats_df.empty:
    # Añadir columnas para cada métrica
    for metric in ['innovacion', 'interdisciplinariedad', 'impacto', 'internacional', 'aplicabilidad']:
        stats_df[metric] = stats_df['stats'].apply(lambda x: x.get(metric, 'N/A') if x else 'N/A')
    
    # Mostrar resumen
    display(stats_df[['universidad', 'programa_count', 'has_metadata', 'innovacion', 'interdisciplinariedad', 'impacto', 'internacional', 'aplicabilidad']])
    
    # Estadísticas sobre universidades con/sin explicaciones
    with_metadata = stats_df[stats_df['has_metadata'] > 0].shape[0]
    total_unis = stats_df.shape[0]
    
    if with_metadata > 0:
        info_msg(f"{with_metadata} de {total_unis} universidades ({with_metadata/total_unis*100:.1f}%) ya tienen explicaciones generadas.")
    else:
        info_msg("Ninguna universidad tiene explicaciones generadas todavía.")
else:
    error_msg("No se encontraron datos de estadísticas para universidades.")

## Generar Explicaciones para las Métricas

Ahora generaremos explicaciones mediante LLM para las métricas académicas de cada universidad. Puedes elegir recalcular todas o sólo algunas métricas específicas.

In [ ]:
# Recalcular todas las métricas y generar explicaciones
results = recalculate_fields()

# Mostrar resumen
if "error" not in results:
    success_msg(f"Proceso completado. Se procesaron {results['universities_processed']} universidades y se actualizaron {results['documents_updated']} documentos.")
    
    if results["errors"]:
        error_msg(f"Se encontraron {len(results['errors'])} errores durante el proceso:")
        for err in results["errors"][:5]:  # Mostrar solo los primeros 5 errores
            print(f"- {err}")
        if len(results["errors"]) > 5:
            print(f"... y {len(results['errors']) - 5} más.")
else:
    error_msg(f"Error al recalcular campos: {results['error']}")

## Visualizar Explicaciones Generadas

Veamos algunas de las explicaciones que se han generado:

In [ ]:
# Obtener todas las explicaciones
explanations = get_university_explanations()

if "error" not in explanations:
    # Mostrar el número de explicaciones obtenidas
    info_msg(f"Se encontraron explicaciones para {len(explanations)} universidades.")
    
    # Crear un DataFrame para visualización
    explanation_data = []
    for uni, data in explanations.items():
        explanation_data.append({
            "universidad": uni,
            "explicacion": data["descripcion"],
            "ultima_actualizacion": data["last_updated"],
            "innovacion": data["stats"].get("innovacion", "N/A"),
            "interdisciplinariedad": data["stats"].get("interdisciplinariedad", "N/A"),
            "impacto": data["stats"].get("impacto", "N/A"),
            "internacional": data["stats"].get("internacional", "N/A"),
            "aplicabilidad": data["stats"].get("aplicabilidad", "N/A")
        })
    
    # Convertir a DataFrame
    exp_df = pd.DataFrame(explanation_data)
    
    # Mostrar algunas explicaciones
    for _, row in exp_df.sample(min(3, len(exp_df))).iterrows():
        display(HTML(f'''
        <div style="border: 1px solid #ddd; padding: 15px; border-radius: 5px; margin: 10px 0;">
            <h3 style="color: #3c40c6;">{row['universidad']}</h3>
            <p><strong>Última actualización:</strong> {row['ultima_actualizacion']}</p>
            <div style="margin: 10px 0; display: flex; flex-wrap: wrap;">
                <div style="background-color: #e3f2fd; padding: 5px 10px; margin: 3px; border-radius: 3px;">Innovación: {row['innovacion']}/10</div>
                <div style="background-color: #e8f5e9; padding: 5px 10px; margin: 3px; border-radius: 3px;">Interdisciplinariedad: {row['interdisciplinariedad']}/10</div>
                <div style="background-color: #fff3e0; padding: 5px 10px; margin: 3px; border-radius: 3px;">Impacto: {row['impacto']}/10</div>
                <div style="background-color: #f3e5f5; padding: 5px 10px; margin: 3px; border-radius: 3px;">Internacional: {row['internacional']}/10</div>
                <div style="background-color: #e0f2f1; padding: 5px 10px; margin: 3px; border-radius: 3px;">Aplicabilidad: {row['aplicabilidad']}/10</div>
            </div>
            <p><strong>Explicación:</strong></p>
            <p>{row['explicacion'].replace('\n', '<br>')}</p>
        </div>
        '''))
else:
    error_msg(f"Error al obtener explicaciones: {explanations['error']}")

## Recalcular Métricas Específicas

También puedes elegir recalcular sólo algunas métricas específicas:

In [ ]:
# Recalcular sólo las métricas seleccionadas
# Opciones: "innovacion", "interdisciplinariedad", "impacto", "internacional", "aplicabilidad"
selected_metrics = ["innovacion", "aplicabilidad"]

# Ejecutar la recalculación
results = recalculate_fields(fields_to_recalculate=selected_metrics)

# Mostrar resumen
if "error" not in results:
    success_msg(f"Proceso completado para las métricas {', '.join(selected_metrics)}. "
                f"Se procesaron {results['universities_processed']} universidades y se actualizaron {results['documents_updated']} documentos.")
else:
    error_msg(f"Error al recalcular campos: {results['error']}")

## Buscar Explicación para una Universidad Específica

In [ ]:
# Nombre de la universidad a buscar
universidad_nombre = "Universidad de Barcelona"  # Cambia esto por el nombre de la universidad que quieras consultar

# Obtener explicación
explicacion = get_university_explanations(university_name=universidad_nombre)

if "error" not in explicacion:
    if explicacion:
        for uni, data in explicacion.items():
            display(HTML(f'''
            <div style="border: 1px solid #ddd; padding: 15px; border-radius: 5px; margin: 10px 0;">
                <h3 style="color: #3c40c6;">{uni}</h3>
                <p><strong>Última actualización:</strong> {data['last_updated']}</p>
                <div style="margin: 10px 0; display: flex; flex-wrap: wrap;">
                    <div style="background-color: #e3f2fd; padding: 5px 10px; margin: 3px; border-radius: 3px;">Innovación: {data['stats'].get('innovacion', 'N/A')}/10</div>
                    <div style="background-color: #e8f5e9; padding: 5px 10px; margin: 3px; border-radius: 3px;">Interdisciplinariedad: {data['stats'].get('interdisciplinariedad', 'N/A')}/10</div>
                    <div style="background-color: #fff3e0; padding: 5px 10px; margin: 3px; border-radius: 3px;">Impacto: {data['stats'].get('impacto', 'N/A')}/10</div>
                    <div style="background-color: #f3e5f5; padding: 5px 10px; margin: 3px; border-radius: 3px;">Internacional: {data['stats'].get('internacional', 'N/A')}/10</div>
                    <div style="background-color: #e0f2f1; padding: 5px 10px; margin: 3px; border-radius: 3px;">Aplicabilidad: {data['stats'].get('aplicabilidad', 'N/A')}/10</div>
                </div>
                <p><strong>Explicación:</strong></p>
                <p>{data['descripcion'].replace('\n', '<br>')}</p>
            </div>
            '''))
    else:
        info_msg(f"No se encontró explicación para la universidad '{universidad_nombre}'.")
else:
    error_msg(f"Error al buscar explicación: {explicacion['error']}")

## Exportar Explicaciones a CSV

Puedes exportar todas las explicaciones a un archivo CSV para análisis o respaldo:

In [ ]:
# Obtener todas las explicaciones
all_explanations = get_university_explanations()

if "error" not in all_explanations:
    # Crear un DataFrame para exportación
    export_data = []
    for uni, data in all_explanations.items():
        export_data.append({
            "universidad": uni,
            "explicacion": data["descripcion"],
            "ultima_actualizacion": data["last_updated"],
            "innovacion": data["stats"].get("innovacion", "N/A"),
            "interdisciplinariedad": data["stats"].get("interdisciplinariedad", "N/A"),
            "impacto": data["stats"].get("impacto", "N/A"),
            "internacional": data["stats"].get("internacional", "N/A"),
            "aplicabilidad": data["stats"].get("aplicabilidad", "N/A")
        })
    
    # Convertir a DataFrame
    export_df = pd.DataFrame(export_data)
    
    # Nombre del archivo con fecha
    from datetime import datetime
    fecha = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"explicaciones_universidades_{fecha}.csv"
    
    # Exportar a CSV
    export_df.to_csv(filename, index=False, encoding='utf-8-sig')
    
    success_msg(f"Explicaciones exportadas correctamente a {filename}")
else:
    error_msg(f"Error al obtener explicaciones para exportar: {all_explanations['error']}")